In [6]:
from pydub import AudioSegment
import numpy as np

audio = AudioSegment.from_mp3("samples/440Hz_44100Hz_16bit_05sec.mp3")
samples = np.array(audio.get_array_of_samples())
audio = AudioSegment.from_mp3("samples/Bas - Tribe with J.Cole.mp3")
samples1 = np.array(audio.get_array_of_samples())
samples,len(samples),samples1,len(samples1)

(array([ 0,  0,  0, ..., 11, 12, 16], dtype=int16),
 222336,
 array([0, 0, 0, ..., 0, 0, 0], dtype=int16),
 22939830)